In [1]:
from sklearn.externals import joblib
from flask import Flask, jsonify,request,json
from flask import render_template
import spacy
import en_core_web_sm
# from googletrans import Translator
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


## chinese word nlp
import jieba.analyse as analyse
import jieba



from sqlalchemy import create_engine
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


C:\Users\Luthe\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [14]:
## DB connection
sql_hostname = 'YOUR_CODE
sql_username = 'YOUR_CODE'
sql_password = 'YOUR_CODE'
sql_main_database = 'YOUR_CODE'
sql_table ="YOUR_CODE"


# engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(sql_username,sql_password,sql_hostname,sql_main_database))

engine = create_engine('mysql://{}:{}@{}/{}?charset=utf8mb4'.format(sql_username,sql_password,sql_hostname,sql_main_database))


## SQL query
query = "select * from " + str(sql_main_database) + "." + str(sql_table)

In [15]:
## read from db
df = pd.read_sql_query(query, engine)


In [6]:
## load NLP model
nlp = en_core_web_sm.load()

## VaderSentiment analyser
analyser = SentimentIntensityAnalyzer()


## user define function

In [22]:



def entity_tag_zh(article):
    entity_list=[]
    for key in analyse.extract_tags(article,3, withWeight=False):  ## second variable refer to total num result return
#     	entity_list.append( key.encode('utf-8'))
        entity_list.append( key)
    return entity_list

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    
    return score



## Name entities tag (chinese)

In [16]:
df.head()

,_id,time,title,desc,source,create_time,query_time
0,1,2020-01-23 10:20:00,四川新增3例输入性新型肺炎病例 累计确诊8例,截止1月23日8时，四川省累计报告新型冠状病毒感染的肺炎确诊病例8例（其中成都4例、绵阳2例...,四川省卫健委,2020-01-23 13:13:18,2020-01-28 18:00:03
1,2,2020-01-23 10:14:00,山东新增4例新型冠状病毒肺炎确诊病例,1月22日0-24时，山东省报告新型冠状病毒感染的肺炎新增确诊病例4例。其中：临沂市首次报告...,健康山东,2020-01-23 13:13:20,2020-01-28 18:00:03
2,3,2020-01-23 10:05:00,澳门确诊第二例新型冠状病毒病例 患者为武汉游客,澳门卫生局表示，澳门确诊第二例新型冠状病毒病例，患者66岁，男性，武汉游客，22日下午入境时...,海外网,2020-01-23 13:13:21,2020-01-28 18:00:03
3,4,2020-01-23 09:12:00,黑龙江确诊首例输入性新型冠状病毒感染病例,23日，黑龙江确诊首例输入性新型冠状病毒感染病例，患者为男性，69岁，牡丹江市人。1月12日...,央视新闻,2020-01-23 13:13:22,2020-01-28 18:00:03
4,5,2020-01-23 08:54:00,上海新增7例新型冠状病毒感染的肺炎确诊病例,上海市卫健委23日通报：1月22日0时—24时，本市新增7例新型冠状病毒感染的肺炎确诊病例，...,央视新闻,2020-01-23 13:13:23,2020-01-28 18:00:03


In [29]:
## apply NER function 
df['entity_zh'] = df['desc'].apply(entity_tag_zh)

In [30]:
df['entity_zh'] 

0          [广安, 成都, 疑似病例]
1          [确诊, 病例, 冠状病毒]
2          [澳门, 确诊, 冠状病毒]
3        [牡丹江市, 冠状病毒, 23]
4         [冠状病毒, 卫健委, 23]
              ...        
164        [出院, 患者, 冠状病毒]
165          [肺炎, 日本, 放送]
166    [出院, 解放军总医院, 冠状病毒]
167     [齐齐哈尔市, 疑似病例, 卫健]
168         [病例, 新增, 新余市]
Name: entity_zh, Length: 169, dtype: object

In [35]:

## split the entity into new row and join with nid colymn
entity_df = (df['entity_zh'].apply(pd.Series)\
            .stack()\
            .reset_index(level=1, drop=True)\
            .to_frame('entity_en')\
            .join(df[['_id','time','title', 'desc'  ,'source']], how='left') )

In [36]:
entity_df

,entity_en,_id,time,title,desc,source
0,广安,1,2020-01-23 10:20:00,四川新增3例输入性新型肺炎病例 累计确诊8例,截止1月23日8时，四川省累计报告新型冠状病毒感染的肺炎确诊病例8例（其中成都4例、绵阳2例...,四川省卫健委
0,成都,1,2020-01-23 10:20:00,四川新增3例输入性新型肺炎病例 累计确诊8例,截止1月23日8时，四川省累计报告新型冠状病毒感染的肺炎确诊病例8例（其中成都4例、绵阳2例...,四川省卫健委
0,疑似病例,1,2020-01-23 10:20:00,四川新增3例输入性新型肺炎病例 累计确诊8例,截止1月23日8时，四川省累计报告新型冠状病毒感染的肺炎确诊病例8例（其中成都4例、绵阳2例...,四川省卫健委
1,确诊,2,2020-01-23 10:14:00,山东新增4例新型冠状病毒肺炎确诊病例,1月22日0-24时，山东省报告新型冠状病毒感染的肺炎新增确诊病例4例。其中：临沂市首次报告...,健康山东
1,病例,2,2020-01-23 10:14:00,山东新增4例新型冠状病毒肺炎确诊病例,1月22日0-24时，山东省报告新型冠状病毒感染的肺炎新增确诊病例4例。其中：临沂市首次报告...,健康山东
...,...,...,...,...,...,...
167,疑似病例,168,2020-01-28 22:54:00,黑龙江28日最新通报：新增1例新型冠状病毒感染的肺炎确诊病例,记者从齐齐哈尔市卫健部门获悉，截至2020年1月28日21：00，齐齐哈尔市新增确诊病例1例...,央视新闻
167,卫健,168,2020-01-28 22:54:00,黑龙江28日最新通报：新增1例新型冠状病毒感染的肺炎确诊病例,记者从齐齐哈尔市卫健部门获悉，截至2020年1月28日21：00，齐齐哈尔市新增确诊病例1例...,央视新闻
168,病例,169,2020-01-29 01:58:00,江西新增37例新型冠状病毒感染的肺炎确诊病例,2020年1月28日0-24时，江西省报告新型冠状病毒感染的肺炎新增确诊病例37例，新增出院...,江西卫健委
168,新增,169,2020-01-29 01:58:00,江西新增37例新型冠状病毒感染的肺炎确诊病例,2020年1月28日0-24时，江西省报告新型冠状病毒感染的肺炎新增确诊病例37例，新增出院...,江西卫健委


In [33]:
entity_df['entity_en'].value_counts()

冠状病毒           72
病例             27
肺炎             12
确诊             12
疫情             11
               ..
18627869526     1
11              1
34              1
荆州市             1
常乐              1
Name: entity_en, Length: 258, dtype: int64

## save result data to db mySQL

In [37]:
entity_df.to_sql(   str((sql_table) +'_nameEntityZH'), con = engine,  if_exists='append',index=False)


In [39]:
entity_df.to_csv(str((sql_table) +'_nameEntityZH'+ '.csv') , index = False)

## sentiment analysis using vaderSentiment ( work well in both formal and informal language (social media）

In [146]:
## save the file to db
df_withSentiment.to_sql(   str((sql_table) +'_sentiment'), con = engine,  if_exists='append',index=False)


In [40]:
df_withSentiment.to_csv(str((sql_table) +'_sentiment'+ '.csv') , index = False)